# Dagum基尼系数分解
## 一、$G_{jj}$的计算
+ 相当于第j个区域内的差异

+ 读入并观察数据

In [1]:
library(dplyr) # distinct 和 count函数
setwd("E:/MyGit/r/Gini")
haven::read_dta("gini.dta") -> df
df |> distinct(id) |> count()

SyntaxError: invalid syntax (2819972759.py, line 3)

+ 先尝试分布计算

In [3]:
df |> dplyr::filter(year == 2005) |> pull(6)  ->  index_2005

purrr::map_dbl(index_2005,~abs(.x - index_2005) |> sum()) |> sum()  ->  delta_2005

index_2005 |> mean() -> mean_2005

delta_2005/(2*mean_2005*279*279)

[1] 0.2004257

+ 定义函数实现上面的操作

In [4]:
# 支持中文变量

G_total <- function(数据,年份) {
  数据 |> dplyr::filter(year == 年份) |> pull(6) -> index
  
  purrr::map_dbl(
    index,
    ~abs(.x - index) |> sum()
  ) |> sum() -> numerator
  
  数据 |> dplyr::filter(year == 年份) |> 
    summarise(
      n = n(),  # 取出个数，这里n==279
      mean = mean(index),
      denominator = 2*n*n*mean
    ) |> pull(denominator) -> denominator
  
  return(numerator/denominator)
}

G_total(df,2005)
G_total(df,2006)

[1] 0.2004257

[1] 0.1380483

In [5]:
purrr::map_df(
  unique(df$year),
  ~tibble(
    year = .x,
    g = G_total(df,.x)
  )
) -> G

G

year,g
<dbl>,<dbl>
2005,0.2004257
2006,0.1380483
2007,0.1447769
2008,0.1461993
2009,0.1535574
2010,0.1435320
2011,0.1497882
2012,0.1470551
2013,0.1308341
